# xENSO


In this tutorial we are going to load SST data from ERSSTv5 to compute different indices used for ENSO monitoring. We will use the following indices:

- Nino 3.4
- ONI
- E and C indices

So we begin by importing the required packages

In [2]:
import cartopy.crs as crrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xeofs as xe

from xenso import indices

Now we load ERSSTv5 data from IRI library. We need to do a bit of cleanup to have nice names and time coordinates.

In [3]:
ersstv5 = (
    xr.open_dataset(
        "http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/dods",
        decode_cf=False,
    )
    .rename(X="lon", Y="lat", T="time")
    .squeeze(drop=True)
)
ersstv5.time.attrs["calendar"] = "360_day"
ersstv5 = xr.decode_cf(ersstv5).sst.convert_calendar("standard", align_on="date").load()
ersstv5

<xarray.DataArray 'sst' (time: 2035, lat: 89, lon: 180)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ...,
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ...,
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ...,
...
        ...,
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ...,
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ...,
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8],
        [-1.8, -1.8, -1.8, ..., -1.8, -1.8, -1.8]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1854-01-16 1854-02-16 ... 2023-07-16
  * lat      (lat) float32 -88.0 -86.0 -84.0 -82.0 -80.0 ... 82.0 84.0 86.0 88.0
Attributes:
    pointwidth:          1.0
    long_name:           Extended reconstructed sea surface temperature
    valid_max:           45.0
    valid_min:           -3.0
    units:               Celsius_scale
    iridl:hasSemantics:  iridl:SeaSurfaceTemperature
    expires:             1693612800
    standard_name:       sea_surface_temperature

Now we are ready to compute the E and C index. We can change the base period to one that fits our need. Since the default value is the one used in the paper when it was released (2011), we are now more than 10 years ahead and the base period can be safely changed to 1991-2020. This base period is used for computing anomalies (in case the data is not an anomaly already) and is also used for computing the climatological EOFs. The E and C index are obtained from the rotation of the projected data onto the climatological EOFs.

In [10]:
ecindex = indices.ECindex(ersstv5, base_period=("1991", "2020"))
ecindex